# Error messages

* Very important
* Have lots of information in them
* Take the time to read them.
* If you get multiple error messages, the first one is the most important, then the last one, then the others in the middle.


An error message has 4 parts:

* The stack trace
* The error itself and a small description (**the most important**) (only 1 line)
* Sometimes: extra details
* Sometimes: hint to help you get more information

Here is an example of code with a user error. Don't try to find the error by code inspection, check only the error:

In [1]:
import numpy as np
import theano
import theano.tensor as T
x = T.vector()
y = T.vector()
z = x + x
z = z * y
f = theano.function([x, y], z)
f(np.ones((2,)), np.ones((3,)))

ValueError: Input dimension mis-match. (input[0].shape[0] = 2, input[1].shape[0] = 3)
Apply node that caused the error: Elemwise{Composite{((i0 + i0) * i1)}}(<TensorType(float64, vector)>, <TensorType(float64, vector)>)
Toposort index: 0
Inputs types: [TensorType(float64, vector), TensorType(float64, vector)]
Inputs shapes: [(2,), (3,)]
Inputs strides: [(8,), (8,)]
Inputs values: [array([ 1.,  1.]), array([ 1.,  1.,  1.])]
Outputs clients: [['output']]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

# Where in your code does this error come from?

In Python, you can pass the parameter `mode=theano.Mode(optimizer=TODO)` to `theano.function`, instead of changing the Theano flag.

This allows you to see in your code, which line caused the problem!

In [2]:
mode = theano.compile.DebugMode()

import numpy as np
import theano
import theano.tensor as T
x = T.vector()
y = T.vector()
z = x + x
z.name = "z1"
z = z * y
z.name = "z2"
f = theano.function([x, y], z, mode=mode)
f(np.ones((2,)), np.ones((3,)))

ValueError: An optimization (probably FusionOptimizer) inserted an apply node that raise an error.
The information we have about this optimizations is:z2
  Elemwise{mul,no_inplace} [@A] 'z2'   
   |Elemwise{add,no_inplace} [@B] 'z1'   
   | |<TensorType(float64, vector)> [@C]
   | |<TensorType(float64, vector)> [@C]
   |<TensorType(float64, vector)> [@D]


The original exception: 
operands could not be broadcast together with shapes (2,) (3,) 
Apply node that caused the error: Elemwise{Composite{((i0 + i0) * i1)}}(<TensorType(float64, vector)>, <TensorType(float64, vector)>)
Toposort index: 0
Inputs types: [TensorType(float64, vector), TensorType(float64, vector)]
Inputs shapes: [(2,), (3,)]
Inputs strides: [(8,), (8,)]
Inputs values: [array([ 1.,  1.]), array([ 1.,  1.,  1.])]
Outputs clients: [['output']]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_

## Stack trace
The stack trace can be very useful. You don't need to understand the part in Theano (where the file is inside `theano/`), just check the part coming from your code files.

# DebugMode

Checks and double-checks everything, extremely slow.

* Compares Python, C and GPU implementations.
* Compares values before and after each optimization.
* By default, raises an error on NaN.
* Sensitive: so frequently reported errors are OK.

Use the Theano flag `mode=DebugMode` or the the parameter `mode=theano.compile.DebugMode()` to `theano.function()`.

## Printing during execution

In [5]:
import theano

x = theano.tensor.vector()
o = theano.printing.Print("a message")(x)
f = theano.function([x], o)

d = f([3, 4])
e = f([1, 2])

a message __str__ = [ 3.  4.]
a message __str__ = [ 1.  2.]


## Printing attributes of a variable

In [6]:
o = theano.printing.Print("Attributes of x:", attrs=('min', 'mean', 'max'))(x)
f = theano.function([x], o)
d = f([3, 1, 4, 9])

Attributes of x: min = 1.0
Attributes of x: mean = 4.25
Attributes of x: max = 9.0


# Most Frequent NaN Causes

* Hyperparameters (ex: learning rate)
* Initialization of parameters
* Numerical Stability
* Algorithm Related

Run in `NanGuardMode`, `DebugMode`, or `MonitorMode`.

# NanGuardMode

Can check for:
* Nan
* Inf
* Big values (greater than 1e10)

In [7]:
import numpy

import theano
import theano.compile.nanguardmode
from theano import tensor as T

x = T.matrix()
w = theano.shared(numpy.random.randn(5, 7).astype(theano.config.floatX))
y = T.dot(x, w)
mode=theano.compile.nanguardmode.NanGuardMode(nan_is_error=True,
                  inf_is_error=True,
                  big_is_error=True)
fun = theano.function(
    [x], y, mode=mode)
infa = numpy.tile(
    (numpy.asarray(100.) ** 1000000), (3, 5))
fun(infa)

ERROR (theano.compile.nanguardmode): Inf detected
ERROR:theano.compile.nanguardmode:Inf detected
ERROR (theano.compile.nanguardmode): Big value detected
ERROR:theano.compile.nanguardmode:Big value detected
ERROR (theano.compile.nanguardmode): In an input
ERROR:theano.compile.nanguardmode:In an input
ERROR (theano.compile.nanguardmode): Inputs: 
ERROR:theano.compile.nanguardmode:Inputs: 
ERROR (theano.compile.nanguardmode): var
ERROR:theano.compile.nanguardmode:var
ERROR (theano.compile.nanguardmode): <TensorType(float64, matrix)>
ERROR:theano.compile.nanguardmode:<TensorType(float64, matrix)>
ERROR (theano.compile.nanguardmode): A. <TensorType(float64, matrix)>
ERROR:theano.compile.nanguardmode:A. <TensorType(float64, matrix)>
ERROR (theano.compile.nanguardmode): val
ERROR:theano.compile.nanguardmode:val
ERROR (theano.compile.nanguardmode): [array([[ inf,  inf,  inf,  inf,  inf],
       [ inf,  inf,  inf,  inf,  inf],
       [ inf,  inf,  inf,  inf,  inf]])]
ERROR:theano.compile.nangua

AssertionError: 
Apply node that caused the error: Dot22(<TensorType(float64, matrix)>, <TensorType(float64, matrix)>)
Toposort index: 0
Inputs types: [TensorType(float64, matrix), TensorType(float64, matrix)]
Inputs shapes: [(3, 5), (5, 7)]
Inputs strides: [(40, 8), (56, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [['output']]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

# Test Value

Give sample values to symbolic variables and have the graph execute
as it is being built. This allows to get some type of error like shape
errors when you build the graph instead of during the execution.


In [8]:
# Can also be 'off', 'ignore', 'raise', 'pdb'
theano.config.compute_test_value = 'warn'

# input which will be of shape (5, 10)
x, y  = T.matrices('xy')

# provide Theano with a default test-value
x.tag.test_value = numpy.random.rand(5, 10)
y.tag.test_value = numpy.random.rand(4, 10)

x + y # warn about the shape error

ValueError: Input dimension mis-match. (input[0].shape[0] = 5, input[1].shape[0] = 4)

# Others

* `theano.tensor.opt.AssertOp`
  * Assertion during execution
* `theano.printing.{debugprint,pydotprint}`
  * Text and graphic display of Theano graphs
* Flag: `profile=True`
  * Profiles the execution time of Theano function
* Flag: `exception_verbosity=True`
  * Extra verbose errors, ex: for `MemoryError`
* Flag: `warn_float64='pdb'`
  * Allows to know where `float64` are build in the graph
  
## Advanced

* Flag: `on_opt_error=True`
  * Makes Theano stop on optimization error instead of skipping the optimization in question
* Flag: `optimizer_verbose=True`
  * Makes Theano print each optimization it applies